<a href="https://colab.research.google.com/github/KaterynaSiniuta/ML-Hillel/blob/main/HW_15_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

# 1. Завантажуємо IMDB датасет
max_features = 10000 # Розмір словника
max_len = 200 # Максимальна довжина рецензії

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

# 2. Функція для створення RNN-моделі
def build_rnn():
    model = keras.Sequential([
        Embedding(max_features, 128, input_length=max_len),
        LSTM(64, return_sequences=True),
        GlobalAveragePooling1D(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# 3. Кількість RNN-моделей в ансамблі
n_models = 5
models = []

for i in range(n_models):
    print(f"Тренування моделі {i+1}/{n_models}...")

    # Вибираємо випадковий піднабір даних (Bootstrap Sampling)
    x_train_sample, y_train_sample = resample(x_train, y_train, n_samples=len(x_train), random_state=i)

    # Створюємо та навчаємо модель
    model = build_rnn()
    model.fit(x_train_sample, y_train_sample, epochs=5, batch_size=128, verbose=1)
    models.append(model)


# 4. Ансамблеве передбачення (Soft Voting)
preds = np.array([model.predict(x_test) for model in models]) # (n_models, n_samples, 1)
avg_preds = np.mean(preds, axis=0) # Усереднюємо ймовірності

# 5. Оцінка точності
y_pred = (avg_preds > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
print(f"Ансамблева точність: {accuracy:.4f}")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Тренування моделі 1/5...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 76s 370ms/step - accuracy: 0.7150 - loss: 0.5442
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 84s 380ms/step - accuracy: 0.9288 - loss: 0.2120
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 83s 384ms/step - accuracy: 0.9599 - loss: 0.1257
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 84s 393ms/step - accuracy: 0.9705 - loss: 0.0973
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 78s 371ms/step - accuracy: 0.9793 - loss: 0.0665
Тренування моделі 2/5...
Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 81s 394ms/step - accuracy: 0.6969 - loss: 0.5547
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 79s 383ms/step - accuracy: 0.9293 - loss: 0.2041
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 83s 384ms/step - accuracy: 0.9600 - loss: 0.1259
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 82s 387ms/step - accuracy: 0.9771 - loss: 0.0777
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 80s 376ms/step - accuracy: 0.9819 - loss: 0.0642
Тренування моделі 3/5...
Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 79s 388ms/step - accuracy: 0.69

ДЗ 15. RNN
**Текст, выделенный полужирным шрифтом**

Для IMDB датасету зробити класифікацію на основі RNN

https://keras.io/api/datasets/imdb/